# Yelp API Calls

Aaron Musselwhite 10/19/22

In [10]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [21]:
!pip install tqdm

     ---------------------------------------- 78.5/78.5 kB 2.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Aaron\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import json
with open('/Users/Aaron/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [23]:
JSON_FILE = "Data/results_in_progress_AZ_Smoked.json"
JSON_FILE

'Data/results_in_progress_AZ_Smoked.json'

In [24]:
file_exists = os.path.isfile(JSON_FILE)
 
if file_exists == False:
    
   
    folder = os.path.dirname(JSON_FILE)
   
    if len(folder)>0:
        
        os.makedirs(folder,exist_ok=True)
        
        
    
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_AZ_Smoked.json not found. Saving empty list to file.


In [35]:
LOCATION = "Tucson, AZ"
TERM = "Smoked Wings"

In [26]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [36]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [37]:
total_results = results['total']
total_results

176

In [38]:
results_per_page = len(results['businesses'])
results_per_page

20

In [39]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

9

In [40]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [42]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/9 [00:00<?, ?it/s]

In [43]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/9 [00:00<?, ?it/s]

In [44]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,WAInJdS3M-4JgUyzf_LoQg,smokey-mo-tucson,Smokey Mo,https://s3-media2.fl.yelpcdn.com/bphoto/L0Jd9s...,False,https://www.yelp.com/biz/smokey-mo-tucson?adju...,533,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 32.2543430452397, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '2650 N 1st Ave', 'address2': '',...",+15202038250,(520) 203-8250,2941.230821
1,QMo3URjgmyY_jU9GtYfxtA,kens-hardwood-barbecue-tucson,Kens Hardwood Barbecue,https://s3-media3.fl.yelpcdn.com/bphoto/39vGA7...,False,https://www.yelp.com/biz/kens-hardwood-barbecu...,259,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 32.20644, 'longitude': -110.87996}","[pickup, delivery]",$$,"{'address1': '5250 E 22nd St', 'address2': '',...",+15207454746,(520) 745-4746,6497.026672
2,pUeuBWxgll2in2KqKeAR_A,firebirds-wood-fired-grill-tucson,Firebirds Wood Fired Grill,https://s3-media2.fl.yelpcdn.com/bphoto/hrkkpD...,False,https://www.yelp.com/biz/firebirds-wood-fired-...,503,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 32.3253079094931, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '2985 E Skyline Dr', 'address2': ...",+15205770747,(520) 577-0747,8917.041611
3,UKImUPq5l6yHn_JO8DQZng,the-neighborhood-tucson,The Neighborhood,https://s3-media2.fl.yelpcdn.com/bphoto/rGI0FZ...,False,https://www.yelp.com/biz/the-neighborhood-tucs...,88,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"{'latitude': 32.1993942260742, 'longitude': -1...",[delivery],$$,"{'address1': '3940 E 29th St', 'address2': Non...",+15202074004,(520) 207-4004,5562.121135
4,G98s4QCjcSRLsr9FSJ7qWQ,the-horseshoe-grill-tucson,The Horseshoe Grill,https://s3-media2.fl.yelpcdn.com/bphoto/jPIgdI...,False,https://www.yelp.com/biz/the-horseshoe-grill-t...,390,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",3.5,"{'latitude': 32.2225669644023, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '7713 E Broadway Blvd', 'address2...",+15208380404,(520) 838-0404,10180.769032


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
171,UnIxj6MCluR-aMuGiree8w,dennys-tucson-7,Denny's,https://s3-media1.fl.yelpcdn.com/bphoto/Pf8T_Z...,False,https://www.yelp.com/biz/dennys-tucson-7?adjus...,58,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.0,"{'latitude': 32.1336674392254, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '2630 E Valencia Rd', 'address2':...",+15202946131,(520) 294-6131,12398.075029
172,qaWM4T1eQoUMSRJ2AL2U1w,red-lobster-oro-valley,Red Lobster,https://s3-media1.fl.yelpcdn.com/bphoto/TbcyR9...,False,https://www.yelp.com/biz/red-lobster-oro-valle...,70,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 32.418604, 'longitude': -110.938204}","[delivery, pickup]",$$,"{'address1': '11695 N Oracle Rd', 'address2': ...",+15205447887,(520) 544-7887,19295.577164
173,X0tlzqEITaegYt10WJkSFA,dennys-oro-valley,Denny's,https://s3-media2.fl.yelpcdn.com/bphoto/M2YWwC...,False,https://www.yelp.com/biz/dennys-oro-valley?adj...,24,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",3.0,"{'latitude': 32.407913, 'longitude': -110.945451}","[delivery, pickup]",$$,"{'address1': '11107 N Oracle Rd', 'address2': ...",+15202690696,(520) 269-0696,18143.173646
174,UILorKaBGq7u_K6wUZKYnA,dennys-tucson-5,Denny's,https://s3-media1.fl.yelpcdn.com/bphoto/u3HNYS...,False,https://www.yelp.com/biz/dennys-tucson-5?adjus...,54,"[{'alias': 'tradamerican', 'title': 'American ...",2.5,"{'latitude': 32.33806085248846, 'longitude': -...","[delivery, pickup]",$,"{'address1': '4920 W Ina Rd', 'address2': '', ...",+15207446805,(520) 744-6805,16578.657596
175,Wcf04Kz9HGlOKazulJ2Jcg,ihop-tucson-9,IHOP,https://s3-media3.fl.yelpcdn.com/bphoto/SS2Zl7...,False,https://www.yelp.com/biz/ihop-tucson-9?adjust_...,100,"[{'alias': 'tradamerican', 'title': 'American ...",2.5,"{'latitude': 32.35982, 'longitude': -111.08498}","[delivery, pickup]",$$,"{'address1': '8445 N Cerius StrAve', 'address2...",+15207441316,(520) 744-1316,19237.507593


In [46]:
final_df.duplicated(subset='id').sum()

0

In [47]:
final_df.to_csv('Data/final_results_AZ_Smoked.csv.gz', compression='gzip',index=False)